<a href="https://colab.research.google.com/github/jongheonleee/Artificial_Intelligence/blob/master/ch16_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
목차
텐서플로로 시퀀스 모델링을 위한 RNN 구현하기
두 번째 프로젝트: 텐서플로로 글자 단위 언어 모델 구현
데이터셋 전처리
문자 수준의 RNN 모델 만들기
평가 단계 - 새로운 텍스트 생성
트랜스포머 모델을 사용한 언어 이해
셀프 어텐션 메카니즘 이해하기
셀프 어텐션 기본 구조
쿼리, 키, 값 가중치를 가진 셀프 어텐션 메카니즘
멀티-헤드 어텐션과 트랜스포머 블록
요약

'''


'\n목차\n텐서플로로 시퀀스 모델링을 위한 RNN 구현하기\n두 번째 프로젝트: 텐서플로로 글자 단위 언어 모델 구현\n데이터셋 전처리\n문자 수준의 RNN 모델 만들기\n평가 단계 - 새로운 텍스트 생성\n트랜스포머 모델을 사용한 언어 이해\n셀프 어텐션 메카니즘 이해하기\n셀프 어텐션 기본 구조\n쿼리, 키, 값 가중치를 가진 셀프 어텐션 메카니즘\n멀티-헤드 어텐션과 트랜스포머 블록\n요약\n\n'

In [2]:
from IPython.display import Image

### 두 번째 프로젝트: 텐서플로로 글자 단위 언어 모델 구현
Image(url='https://git.io/JLdVE', width=700)

In [3]:
### 데이터셋 전처리

# 코랩에서 실행할 경우 다음 코드를 실행해 주세요.
!wget https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch16/1268-0.txt

--2022-08-29 11:31:06--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch16/1268-0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1171600 (1.1M) [text/plain]
Saving to: ‘1268-0.txt’

1268-0.txt          100%[===================>]   1.12M  --.-KB/s    in 0.05s   

2022-08-29 11:31:06 (21.2 MB/s) - ‘1268-0.txt’ saved [1171600/1171600]



In [4]:
import numpy as np

with open('1268-0.txt', 'r', encoding='UTF8') as fp:
  text = fp.read()

start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
print(start_indx, end_indx)

text = text[start_indx:end_indx]
char_set = set(text)

print('전체 길이:', len(text))
print('고유한 문자:', len(char_set))

567 1112917
전체 길이: 1112350
고유한 문자: 80


In [5]:
Image(url='https://git.io/JLdVz', width=700)

In [6]:
chars_sorted = sorted(char_set)
char2int = {ch: i for i, ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32
)

print('인코딩된 텍스트 크기:', text_encoded.shape)

print(text[:15], '     ==인코딩==>', text_encoded[:15])
print(text_encoded[15:21], ' ==디코딩==>', ''.join(char_array[text_encoded[15:21]]))

인코딩된 텍스트 크기: (1112350,)
THE MYSTERIOUS       ==인코딩==> [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]
[33 43 36 25 38 28]  ==디코딩==> ISLAND


In [7]:
Image(url='https://git.io/JLdVV', width=700)

In [8]:
import tensorflow as tf


ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
  print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


In [9]:
seq_length = 40
chunk_size = seq_length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

print(ds_chunks)

## inspection:
for seq in ds_chunks.take(1):
  input_seq = seq[:seq_length].numpy()
  target = seq[seq_length].numpy()
  print(input_seq, ' ->', target)
  print(repr(''.join(char_array[input_seq])),
        ' ->', repr(''.join(char_array[target])))

<BatchDataset element_spec=TensorSpec(shape=(41,), dtype=tf.int32, name=None)>
[44 32 29  1 37 48 43 44 29 42 33 39 45 43  1 33 43 36 25 38 28  1  6  6
  6  0  0  0  0  0 40 67 64 53 70 52 54 53  1 51]  -> 74
'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'  -> 'y'


In [10]:
Image(url='https://git.io/JLdVr', width=700)

In [11]:
# x & y를 나누기 위한 함수를 정의함
def split_input_target(chunk):
  input_seq = chunk[:-1]
  target_seq = chunk[1:]

  return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

for example in ds_sequences.take(2):
  # print(example)
  print('입력 (x):', repr(''.join(char_array[example[0].numpy()])))
  print('타깃 (y):', repr(''.join(char_array[example[1].numpy()])))
  print()

입력 (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
타깃 (y): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'

입력 (x): ' Anthony Matonak, and Trevor Carlson\n\n\n\n'
타깃 (y): 'Anthony Matonak, and Trevor Carlson\n\n\n\n\n'



In [12]:
# 배치 크기
BATCH_SIZE = 64
BUFFER_SIZE = 10000

tf.random.set_seed(1)
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 40), dtype=tf.int32, name=None), TensorSpec(shape=(None, 40), dtype=tf.int32, name=None))>

In [13]:
# 문자 수준의 RNN 모델 만들기


def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(
            rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size = charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         20480     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1574912   
                                                                 
 dense (Dense)               (None, None, 80)          41040     
                                                                 
Total params: 1,636,432
Trainable params: 1,636,432
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    )
)

model.fit(ds, epochs=20)

Epoch 1/20
424/424 [==============================] - 408s 944ms/step - loss: 2.3212
Epoch 2/20
424/424 [==============================] - 401s 944ms/step - loss: 1.7581
Epoch 3/20
424/424 [==============================] - 404s 952ms/step - loss: 1.5524
Epoch 4/20
424/424 [==============================] - 455s 1s/step - loss: 1.4330
Epoch 5/20
424/424 [==============================] - 426s 1s/step - loss: 1.3577
Epoch 6/20
424/424 [==============================] - 431s 1s/step - loss: 1.3056
Epoch 7/20
424/424 [==============================] - 471s 1s/step - loss: 1.2657
Epoch 8/20
424/424 [==============================] - 469s 1s/step - loss: 1.2340
Epoch 9/20
424/424 [==============================] - 472s 1s/step - loss: 1.2086
Epoch 10/20
424/424 [==============================] - 458s 1s/step - loss: 1.1860
Epoch 11/20
424/424 [==============================] - 424s 999ms/step - loss: 1.1666
Epoch 12/20
 74/424 [====>.........................] - ETA: 5:45 - loss: 1.1586

In [15]:
# 평가 단계 - 새로운 텍스트 생성

tf.random.set_seed(1)

logits = [[1.0, 1.0, 1.0]]
print('확률:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10
)

tf.print(samples.numpy())

확률: [0.33333334 0.33333334 0.33333334]
array([[0, 0, 1, 2, 0, 0, 0, 0, 1, 0]])


In [16]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 3.0]]
print('확률:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10
)
tf.print(samples.numpy())


확률: [0.10650698 0.10650698 0.78698605]
array([[2, 0, 2, 2, 2, 0, 1, 2, 2, 0]])


In [17]:
def sample(model, starting_str, 
           len_generated_text=500,
           max_input_length=40,
           scale_factor=1.0):
  encoded_input = [char2int[s] for s in starting_str]
  encoded_input = tf.reshape(encoded_input, (1, -1))

  generated_str = starting_str

  model.reset_states()
  for i in range(len_generated_text):
    logits = model(encoded_input)
    logits = tf.squeeze(logits, 0)

    scaled_logits = logits * scale_factor
    new_char_indx = tf.random.categorical(
        scaled_logits, num_samples=1
    )

    new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()

    generated_str += str(char_array[new_char_indx])

    new_char_indx = tf.expand_dims([new_char_indx], 0)
    encoded_input = tf.concat(
        [encoded_input, new_char_indx],
        axis = 1
    )
    encoded_input = encoded_input[:, -max_input_length:]

  return generated_str

tf.random.set_seed(1)
print(sample(model, starting_str='The island'))


The island may be began to communicate thirty earth to exist in the corral. To make them served as from the mouth of the island. In this degient had become very used for cementation. They must then we have no
matter of stea, merimes, for there was nothing to defend more than the corral.”

“That is there!” said Cyrus Harding, “back one prove at the try,” asked Herbert. “How will friend down, thir branches were kelighing. Also that therefore, before grew two days
aloped upon him. The upper water appeared to


In [ ]:
# 예측 가능성 대 무작위성

logits = np.array([[1.0, 1.0, 3.0]])

print('스케일 조정 전의 확률:', tf.math.softmax(logits).numpy()[0])
print('0.5배 조정 후 확률:', tf.math.softmax(0.5*logits).numpy()[0])
print('0.1배 조정 후 확률:', tf.math.softmax(0.1*logits).numpy()[0])

In [19]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island',
             scale_factor=0.5))


The island had occarcadness obliquesion, throm
Clan
pideo--amone,-fullows, oryajenquib upon the Heighmoom.

Oncheasides, grouse. He termed, angxied
he applishererso,
inStead draw over. Grong Herald.”

This oughly understood
did these imagination. Where.”

“Did kepteom, criking. Feor drox cone eromitions), livea, pheoc! Ribod la-: Aurtestit,
Cyrus,-vizent
when
He hinked, without human
inlriseply.

The poorliegh! HeBberm:”

“Thodams?” exsmopo: Halbon.”
sapth?

Cabfort outen, and princoppie.”

Loich Bobtion 


In [20]:
'''
트랜스포머 모델을 사용한 언어 이해
셀프 어텐션 메카니즘 이해하기
셀프 어텐션 기본 구조
'''
Image(url='https://git.io/JLdVo', width=700)

In [21]:
'''
쿼리, 키, 값 가중치를 가진 셀프 어텐션 메카니즘
멀티-헤드 어텐션과 트랜스포머 블록
'''

Image(url='https://git.io/JLdV6', width=700)